In [1]:
from wikidata.utils import get_label, load_json



In [2]:
from query import Query
from pathlib import Path
from wikidata.utils import get_label, load_json
facts_fname = '/Users/annabelle/workplace/RippleEdits/data/benchmark/top_ents_5_entities_0_facts_each.json'
facts = load_json(facts_fname)

In [6]:
from wikidata.relations import our_relations, relation2impacted_relations, relation2phrase
from wikidata.utils import subject_relation_to_targets, ent_to_relation_ids, get_label, get_aliases, get_description, \
    subjects_given_relation_target
from build_logical_constraints import generate_constraints
from utils import create_test_example_given_input_targets
from relation import Relation
from query import Query, TwoHopQuery
from testcase import TestCase
from two_hop_phrases import relation_couple_to_phrase

def update_facts(facts, s, r, o):
    f_string = f'{str(s)}_{str(r.id())}_{str(o)}'
    if f_string not in facts:
        facts[f_string] = Query(s, r, o)

def forward_two_hop_axis(subject_id: str, relation: Relation, target_id: str, facts: dict):
    tests = []
    if not target_id or target_id[0] != 'Q':
        return tests
    for backward_relation in Relation:
        backward_relation_id = backward_relation.id()
        backward_subjects = subjects_given_relation_target(backward_relation_id, subject_id)
        for backward_subject in backward_subjects:
            phrase = relation_couple_to_phrase(backward_relation, relation)
            if phrase is None:
                continue
            phrase = phrase.replace('<subject>', get_label(backward_subject))
            update_facts(facts, backward_subject, backward_relation, subject_id)
            update_facts(facts, subject_id, relation, target_id)
            tests.append(TwoHopQuery(backward_subject, backward_relation, subject_id, relation, target_id, phrase))
    return tests

def two_hop_axis(subject_id: str, relation: Relation, target_id: str, facts: dict):
    tests = []
    if not target_id or target_id[0] != 'Q':
        return tests
    target_relations = ent_to_relation_ids(target_id)
    for relation_id in target_relations:
        second_relation_enum = Relation.id_to_enum(relation_id)
        if second_relation_enum is None:
            continue
        second_hop_targets = subject_relation_to_targets(target_id, second_relation_enum)
        for second_hop_target in second_hop_targets:
            phrase = relation_couple_to_phrase(relation, second_relation_enum)
            if phrase is None:
                continue
            phrase = phrase.replace('<subject>', get_label(subject_id))
            test_query = TwoHopQuery(subject_id, relation, target_id, second_relation_enum, second_hop_target, phrase)
            update_facts(facts, target_id, second_relation_enum, second_hop_target)
            update_facts(facts, subject_id, relation, target_id)
            tests.append(TwoHopQuery(subject_id, relation, target_id, second_relation_enum, second_hop_target, phrase))
    return tests


In [9]:
updated_facts = {}
two_hop_all = []
for i, f in enumerate(facts):
    query = facts[f]
    q = Query.from_dict(query)
    print(f'At fact {i}')
    for target in q._targets_ids:
        
        two_hop = two_hop_axis(q._subject_id, q._relation, target, updated_facts)
        forward_two_hop = forward_two_hop_axis(q._subject_id, q._relation, target, updated_facts)
        two_hop_all = two_hop_all  + forward_two_hop + two_hop
        print(f'two hop {two_hop_all}')
        print(f'Len two hop {len(two_hop_all)}')
        print(f'len facts {len(updated_facts.keys())}')
        if len(two_hop_all) > 100:
            break
    else:
        continue
    break

At fact 0
two hop [<query.TwoHopQuery object at 0x7fe05ebc5e50>, <query.TwoHopQuery object at 0x7fe05eb59970>, <query.TwoHopQuery object at 0x7fe09a0dc070>, <query.TwoHopQuery object at 0x7fe09a105d90>, <query.TwoHopQuery object at 0x7fe05eb80970>, <query.TwoHopQuery object at 0x7fe05eb80c40>, <query.TwoHopQuery object at 0x7fe05eb80e20>, <query.TwoHopQuery object at 0x7fe0386c85e0>, <query.TwoHopQuery object at 0x7fe0386c8130>, <query.TwoHopQuery object at 0x7fe0386c8a00>, <query.TwoHopQuery object at 0x7fe0386c8040>, <query.TwoHopQuery object at 0x7fe0386c8b80>, <query.TwoHopQuery object at 0x7fe0386c86d0>, <query.TwoHopQuery object at 0x7fe0386c8d00>, <query.TwoHopQuery object at 0x7fe0386c8910>, <query.TwoHopQuery object at 0x7fe0386c8280>, <query.TwoHopQuery object at 0x7fe0386c8700>]
Len two hop 17
len facts 18
At fact 1
two hop [<query.TwoHopQuery object at 0x7fe05ebc5e50>, <query.TwoHopQuery object at 0x7fe05eb59970>, <query.TwoHopQuery object at 0x7fe09a0dc070>, <query.TwoHopQ

In [18]:
import os
facts_file='top_ents_10_entities_0_facts_each.json'
DATA_DIR = '/Users/annabelle/workplace/RippleEdits/data/benchmark'
fname = os.path.join(DATA_DIR, facts_file)
facts = load_json(fname)
list(facts.keys())


['Q322063_P21_Q6581097',
 'Q322063_P106_Q3665646',
 'Q322063_P54_Q132893',
 'Q322063_P54_Q162990',
 'Q322063_P54_Q168506',
 'Q322063_P54_Q131371',
 'Q322063_P54_Q235326',
 'Q322063_P54_Q7401513',
 'Q322063_P19_Q61',
 'Q322063_P27_Q30',
 'Q322063_P69_Q14691919',
 'Q322063_P69_Q1475413',
 'Q322063_P118_Q842793',
 'Q322063_P118_Q94861615',
 'Q322063_P118_Q155223',
 'Q234109_P21_Q6581072',
 'Q234109_P19_Q216638',
 'Q234109_P20_Q724182',
 'Q234109_P69_Q1753535',
 'Q234109_P69_Q3228965',
 'Q234109_P69_Q7961053',
 'Q234109_P27_Q145',
 'Q234109_P106_Q1281618',
 'Q234109_P106_Q1028181',
 'Q234109_P106_Q15296811',
 'Q234109_P106_Q33231',
 'Q234109_P106_Q483501',
 'Q234109_P166_Q12201434',
 'Q234109_P166_Q12201477',
 'Q234109_P26_Q281637',
 'Q234109_P26_Q3182465',
 'Q234109_P101_Q11634',
 'Q234109_P172_Q842438',
 'Q36215_P21_Q6581097',
 'Q36215_P19_Q462177',
 'Q36215_P106_Q5482740',
 'Q36215_P106_Q131524',
 'Q36215_P106_Q82594',
 'Q36215_P106_Q15472169',
 'Q36215_P106_Q484876',
 'Q36215_P106_Q123